In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Cargar el dataset
data = pd.read_csv("Temp_Asu20092021.csv")

# Convertir la columna 'Fecha' a tipo datetime
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Establecer la columna 'Fecha' como índice
data.set_index('Fecha', inplace=True)

# Filtrar datos desde 2019 en adelante
data = data[data.index.year >= 2019]

# Eliminar filas con valores faltantes
data.dropna(inplace=True)

# Crear características derivadas
data['Hora'] = data.index.hour
data['Dia'] = data.index.dayofweek  
data['Mes'] = data.index.month

# Usar características relevantes para la predicción
features = ['Temperatura','Humedad']

# Desplazar la variable objetivo (Temperatura)
salida = data[['Temperatura']].shift(-1)

# Definir características (features) y variable objetivo (target)
X = data[features].values[:-1]  # Incluye las características
y = salida.values[:-1]

# Normalizar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y validación
train_indices = (data.index.year <= 2020)[:-1]
val_indices = (data.index.year == 2021)[:-1]

X_train, X_val = X_scaled[train_indices], X_scaled[val_indices]
y_train, y_val = y[train_indices], y[val_indices]

# Definir el modelo de regresión Ridge
ridge = Ridge()

# Definir la cuadrícula de parámetros para GridSearch
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]  # Regularización L2
}

# Usar GridSearchCV para buscar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:")
print(grid_search.best_params_)

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de validación
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)

# Calcular el MSE para la temperatura
mse_temp = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error para la Temperatura en validación: {mse_temp}")
print(f"Raíz del Mean Squared Error para la Temperatura: {np.sqrt(mse_temp)}")

# Imprimir resultados de predicción
results = pd.DataFrame({
    'Temperatura Real': y_val.flatten(),
    'Predicción Temperatura': y_pred.flatten()
}, index=data.index[1:][val_indices])  # Asegúrate de ajustar el índice correctamente

print("\nResultados de predicción:")
print(results)




Fitting 3 folds for each of 5 candidates, totalling 15 fits
Mejores hiperparámetros:
{'alpha': 0.01}
Mean Squared Error para la Temperatura en validación: 14.951856408959385
Raíz del Mean Squared Error para la Temperatura: 3.8667630401874105

Resultados de predicción:
                     Temperatura Real  Predicción Temperatura
Fecha                                                        
2021-01-01 03:00:00         21.876715               24.626855
2021-01-01 06:00:00         20.676154               22.290293
2021-01-01 09:00:00         19.246456               21.205731
2021-01-01 12:00:00         24.903557               19.799599
2021-01-01 15:00:00         33.581927               24.393599
...                               ...                     ...
2021-12-31 09:00:00         21.719499               23.389467
2021-12-31 12:00:00         29.548918               22.639748
2021-12-31 15:00:00         40.026148               28.930903
2021-12-31 18:00:00         41.244712            